In [34]:
### preparing model for extracted features from the payloads

import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier


payloads_with_feature_df = pd.read_csv("build/combined_payloads_with_features.csv" , index_col=0)
payloads_df = pd.read_csv("build/combined_payloads.csv" , index_col=0)


In [35]:
#dropping payload column

payloads_with_feature_df.drop(["payloads"],axis=1,inplace=True)

payloads_df

,payloads,is_malicious
0,Nokia,0
1,Sony,0
2,BMW,0
3,Palm,0
4,Adobe,0
...,...,...
41575,<form><button formaction=javascript&colon;aler...,1
41576,"<math><a xlink:href=""//jsfiddle.net/t846h/"">click",1
41577,"<object data=data:text/html;base64,PHN2Zy9vbmx...",1
41578,"<iframe src=""data:text/html,%3C%73%63%72%69%70...",1


In [36]:
x_train , x_test , y_train , y_test = train_test_split(payloads_with_feature_df.drop(["is_malicious"],axis=1).values 
                                                ,payloads_with_feature_df["is_malicious"].values , random_state=0 )



x_test

array([[ 98,   0,  36, ...,   0,  16,   0],
       [ 53,   0,  22, ...,   0,   6,   0],
       [ 52,   0,  21, ...,   0,  20,   0],
       ...,
       [108,   0,  19, ...,   0,  19,   0],
       [ 32,   0,   8, ...,   0,   8,   0],
       [ 25,   0,  16, ...,   0,   0,   0]])

In [22]:
svc_model = SVC()

svc_model.fit(x_train,y_train)

SVC()

In [23]:
svc_model.score(x_test,y_test)

0.9975949975949976

In [37]:
# appending path for importing common process ua

import sys 
sys.path.append('/home/th3h04x/Documents/github/ProjectHeatBlast')



In [47]:
# nltk models using count vectorizer

from ReverseProxy.utils.util import process_ua

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix
#save the model
import pickle , joblib

payloads_df.dropna(inplace=True)

x_train , x_test , y_train , y_test = train_test_split(payloads_df["payloads"].values ,payloads_df['is_malicious'].values , test_size=0.2) 


from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
cv = CountVectorizer(analyzer=process_ua)





In [51]:

from sklearn.pipeline import Pipeline , make_pipeline

knn_cv_pipe = Pipeline([("count-vectorizer" , cv) , ("k-nearest neighbours" ,KNeighborsClassifier(n_neighbors=100)) ])
mnb_cv_pipe = Pipeline([("count-vectorizer" ,cv ), ("Multinomial-Naive-Bayes" ,MultinomialNB(alpha=0.5)) ])
svc_cv_pipe = Pipeline([("count-vectorizer" , cv ), ("Support-Vector-Classifier" ,SVC(kernel='sigmoid', gamma=1.0)) ])
dtc_cv_pipe = Pipeline([("count-vectorizer" , cv ), ("Decision-Tree-Classifier" ,DecisionTreeClassifier(min_samples_split=7, random_state=252)) ])
rf_cv_pipe = Pipeline([("count-vectorizer" ,cv ), ("RandomForest-Classifier" ,RandomForestClassifier(n_estimators=37, random_state=252)) ])
lg_cv_pipe = Pipeline([("count-vectorizer" , cv ), ("Logistic-Regression" ,LogisticRegression()) ])

knn_cv_pipe.fit(x_train , y_train)
mnb_cv_pipe.fit(x_train , y_train)
svc_cv_pipe.fit(x_train , y_train)
dtc_cv_pipe.fit(x_train , y_train)
rf_cv_pipe.fit(x_train,y_train)
lg_cv_pipe.fit(x_train,y_train)

Pipeline(steps=[('count-vectorizer',
                 CountVectorizer(analyzer=<function process_ua at 0x7f2f80763160>)),
                ('Logistic-Regression', LogisticRegression())])

In [52]:
#store the count vectorizer models 


file_path = "../blasterModels/payloads_models"

def dump(model , filename):
    with open(filename, "wb") as f:
        joblib.dump(model , f)


dump(knn_cv_pipe , filename=f"{file_path}/knn_cv_pipe.pkl")
dump(mnb_cv_pipe , filename=f"{file_path}/mnb_cv_pipe.pkl")
dump(svc_cv_pipe , filename=f"{file_path}/svc_cv_pipe.pkl")
dump(dtc_cv_pipe , filename=f"{file_path}/dtc_cv_pipe.pkl" )
dump(rf_cv_pipe , filename=f"{file_path}/rf_cv_pipe.pkl")
dump(lg_cv_pipe , filename=f"{file_path}/lg_cv_pipe.pkl")

y_knn = knn_cv_pipe.predict(x_test)
y_mnb = mnb_cv_pipe.predict(x_test)
y_svc = svc_cv_pipe.predict(x_test)
y_dtc = dtc_cv_pipe.predict(x_test)
y_rfc = rf_cv_pipe.predict(x_test)
y_lg = lg_cv_pipe.predict(x_test)

print("[+] ACCURACY \n")
print("Kneighrest neighbours : " , accuracy_score(y_test , y_knn))
print("Multinomial Naive Bayes : " , accuracy_score(y_test , y_mnb))
print("Support Vector Classifier : " , accuracy_score(y_test , y_svc))
print("Decision Tree Classifier : " , accuracy_score(y_test , y_dtc))
print("Random Forest Classifier : " , accuracy_score(y_test , y_rfc))
print("Logistic Regression : " , accuracy_score(y_test , y_lg))




[+] ACCURACY 

Kneighrest neighbours :  0.7663540163540163
Multinomial Naive Bayes :  0.9344636844636844
Support Vector Classifier :  0.9213564213564214
Decision Tree Classifier :  0.5791245791245792
Random Forest Classifier :  0.5751563251563252
Logistic Regression :  0.8664021164021164
